# Example notebook for loading TOFWERK HD5 dataset

Load the FIB image and peak data

In [ ]:
import twtof
import xarray as xr
import pint_xarray
import matplotlib.pyplot as plt


def xrshow(data):
    """Display a xarray
    
    Parameters
    ----------
    data: xarray.DataArray
        Input 2D data 
    """
    img = data.pint.dequantify(format="~P")
    plt.imshow(
        img,
        extent=(0, img["X"][-1], 0, img["Y"][-1]),
        cmap="gray",
    )
    plt.xlabel(f"X [{img.X.units}]")
    plt.ylabel(f"Y [{img.Y.units}]")

# Define the name of the file
filename = "./data/20230722_magnetotactic_HO027_1_Au_pos_spot2_run1_30kV_50pA.h5"
filename = "./data/20241017_Fluorine_HO_134_1_spot8_run1_30kV_50pA.h5"

# Load the FIB image and the peak data
fib, peak = twtof.imread(filename)
print(f"Filename : {filename}")
print(f"Loaded FIB image data with shape {fib.shape}")
print(f"Loaded peak data with shape {peak.shape}")

# Display the two images
plt.subplot(121)
xrshow(fib[10])
plt.title("FIB")
plt.subplot(122)
xrshow(peak[10, 10])
plt.title("Peak at m/c:10, and slice: 10]");

Interactive display of the image and the associated spectrum

In [ ]:
%matplotlib ipympl
from ipywidgets import interact
import matplotlib.pyplot as plt
import numpy as np

d = int(fib.shape[-1] / peak.shape[-1])
pmax = peak[1:].max()
mass, z, y, x = 25, 4, fib.shape[1] // 2, fib.shape[2] // 2
# index in peak first (mass) axis for the mass/charge of interest
mass_idx = np.where(np.abs(peak.mass.to_numpy() - 23) < 0.1)[0]
fig, ax = plt.subplots(1, 3, figsize=(10, 5))
axfib = ax[0].imshow(fib[0].squeeze())
crosshair = ax[0].plot(0, 0, "w+", ms=50)
axpeak = ax[1].plot(peak.mass, peak[:, z, y // d, x // d].to_numpy().squeeze())
sline = ax[1].plot((mass, mass), (0, pmax), "r-")
axmap = ax[2].imshow(peak[mass_idx, z, :, :].squeeze(), vmin=0, vmax=0.01)
ax[0].set_axis_off()
ax[0].set_title(f"FIB Image\n({x},{y},{z})")
ax[1].set(xlabel="mass/charge [m/Q]", ylabel="ion/extraction", ylim=[0, 0.01])
ax[1].set_box_aspect(1)
ax[1].set_title(f"Mass Spectra")
ax[2].set_axis_off()
ax[2].set_title(f"Peak data for mass/charge={mass}, slice={z}")
plt.tight_layout()


@interact(
    x=(0, fib.shape[2] - 1, 1),
    y=(0, fib.shape[1] - 1, 1),
    z=(0, fib.shape[0] - 1, 1),
    mass=(1, peak.shape[3], 1),
    vmax=(0.00001, 1.0, 0.0001),
    cmap=["viridis", "jet", "gray"],
)
def display_twotof(x, y, z, mass, vmax, cmap):
    axfib.set(data=fib[z], cmap=cmap)
    crosshair[0].set(data=([x], [y]))
    ax[0].set_title(f"FIB Image ({x},{y},{z})")
    axpeak[0].set_data(
        (peak.mass.to_numpy(), peak[:, z, x // d, y // d].to_numpy().squeeze())
    )
    sline[0].set(data=((mass, mass), (0, peak[1:].max())))
    ax[1].set_ylim((0, np.pow(vmax, 4)))
    ax[1].set_title(f"Mass Spectra")
    mass_idx = np.where(np.abs(peak.mass.to_numpy() - mass) < 0.1)[0]
    axmap.set_data(peak[mass_idx, z, :, :].squeeze())
    axmap.set_clim(vmin=0, vmax=np.pow(vmax, 4))
    ax[2].set_title(f"Peak data for m/c={mass}, slice={z} ")

Load the peak table which provide details on the mass measurement such as integration limits.

In [ ]:
with twtof.TofH5Reader(filename) as f:
    peak_df = f.load_peak_table()
peak_df

Load all the data in a dictionnary

In [ ]:
data = twtof.as_dict(filename)
data["Acquisition log"]

In [ ]:
twtof.print_content(filename)